# Value at risk - monte carlo

In this notebook, we show how to
- retrieve all models trained in previous stage
- compute covariance matrix of market factor returns
- create a multivariate normal distribution that simulates our market conditions
- run that experiment and store on delta

# `STEP0` Control parameters

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import mlflow
import mlflow.sklearn
from datetime import datetime, timedelta
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
portfolio_table = 'var_portfolio'
stock_table = 'var_stock'
stock_return_table = 'var_stock_return'
market_table = 'var_market'
market_return_table = 'var_market_return'
trial_table = 'var_monte_carlo'

# when do we want to simulate data
trial_date = datetime.strptime('2019-08-01', '%Y-%m-%d')

# where did we log our model
model_path = '/dbfs/databricks/mlflow/4019741983524247/fa5ac8346be649b48730e96c10e1243f/artifacts/models.json'

# how much history do we want compute volatility from
d_days = 90

# how many simulations do we want to run (industry standard ~ 20,000)
runs = 50000

# how many executors can run in parallel
parallelism = 12

# our predictive market factors
feature_names = ['SP500', 'NYSE', 'OIL', 'TREASURY', 'DOWJONES']

# `STEP1` Retrieve models and data

In [0]:
# models we serialized as json from pandas dataframe
# we load models as dictionary of instrument <-> weights
models = {}
for model in np.array(pd.read_json(model_path)):
  models[model[0]] = model[1]
  
model_dict = spark.sparkContext.broadcast(models)

In [0]:
def retrieve_market_factors(from_date, to_date):
  
  # Retrieve market factor returns in the provided time window
  from_ts = F.to_date(F.lit(from_date)).cast(TimestampType())
  to_ts = F.to_date(F.lit(to_date)).cast(TimestampType())
  f_ret = spark.table(market_return_table) \
    .filter(F.col('date') > from_ts) \
    .filter(F.col('date') <= to_ts) \
    .orderBy(F.asc('date')) \
    .dropna()

  # Market factors easily fit in memory and will be used to create multivariate distribution of normal returns
  f_ret_pdf = f_ret.toPandas()
  f_ret_pdf.index = f_ret_pdf['date']
  f_ret_pdf = f_ret_pdf.drop(['date'], axis=1)
  return f_ret_pdf

# SAMPLE DATA
to_date = (datetime.now()).strftime("%Y-%m-%d")
from_date = (datetime.now() - timedelta(days = d_days)).strftime("%Y-%m-%d")
market_factors_df = retrieve_market_factors(from_date, to_date)
market_factors_df.head(10)

,SP500,NYSE,OIL,TREASURY,DOWJONES
date,,,,,
2020-02-13,0.001632,0.002687,0.007109,0.008007,0.004345
2020-02-14,-0.001842,0.000121,0.007248,0.018097,0.000858
2020-02-17,0.000000,0.000000,0.000000,0.000000,0.000000
2020-02-18,0.002924,0.004146,0.005921,0.020357,0.005659
2020-02-19,-0.004695,-0.003422,-0.017316,-0.008957,-0.003955
2020-02-20,0.003823,0.001822,0.000220,0.029081,0.004373
2020-02-21,0.010574,0.006113,0.013359,0.036052,0.007819
2020-02-24,0.034088,0.032112,0.051228,0.066035,0.036231
2020-02-25,0.030748,0.029269,0.042099,0.034728,0.031958


# `STEP2` Generate market conditions

In [0]:
def compute_volatility(f_ret_pdf):
  # Retrieve market factor covariance matrix and average of returns
  # This will be used to generate a multi variate distribution of market factor returns
  return np.array(f_ret_pdf.mean()), np.array(f_ret_pdf.cov())

f_ret_avg, f_ret_cov = compute_volatility(market_factors_df)
f_ret_avg

Out[34]: array([0.00226351, 0.00358161, 0.00596641, 0.01283786, 0.00315668])

In [0]:
# generate same feature vectors as used at model training phase
# add non linear transformations as simple example on non linear returns
def featurize(xs):
  fs = []
  for x in xs:
    fs.append(x)
    fs.append(np.sign(x) * x**2)
    fs.append(x**3)
    fs.append(np.sign(x) * np.sqrt(abs(x)))
  return fs

# provided covariance matrix and average of market factor, we sample from a multivariate distribution
# we allow a seed to be passed for reproducibility
def simulate_market(f_ret_avg_pdf, f_ret_cov_pdf, seed):
  np.random.seed(seed = seed)
  return np.random.multivariate_normal(f_ret_avg_pdf, f_ret_cov_pdf)

# provided simulated market factors and a specific model for an instrument,
# predict the instrument return in 2 weeks time
def predict(fs, ps):
  s = ps[0]
  for i, f in enumerate(fs):
    s = s + ps[i + 1] * f
  return float(s)

In [0]:
seed_init = 42
seeds = [seed_init + x for x in np.arange(0, 10)]
conditions = []
for seed in seeds:
  conditions.append(simulate_market(f_ret_avg, f_ret_cov, seed))

df = pd.DataFrame(conditions, columns=feature_names)
df['_seed'] = seeds
df

,SP500,NYSE,OIL,TREASURY,DOWJONES,_seed
0,0.002876,-0.005541,-0.018411,-0.050048,0.001219,42
1,0.011545,0.019104,0.044907,-0.047773,0.018623,43
2,0.002825,0.004074,-0.049632,0.143556,-0.002633,44
3,-0.007891,-0.005829,0.000211,0.019150,-0.012171,45
4,-0.037699,-0.036611,-0.088991,-0.013229,-0.038103,46
5,0.005736,-0.002550,-0.038707,0.154807,-0.001075,47
6,0.051305,0.050828,0.105587,0.091738,0.053879,48
7,0.064828,0.054320,0.072210,0.104370,0.061417,49
8,0.033461,0.045337,0.078695,0.192836,0.042228,50
9,0.018778,0.022263,-0.011742,0.048931,0.021885,51


# `STEP3` Run monte-carlo

In [0]:
@pandas_udf('ticker string, seed int, trial float', PandasUDFType.GROUPED_MAP)
def run_trials(pdf):
  
  # Deserialize objects from cache
  models = model_dict.value
  f_ret_avg = f_ret_avg_B.value
  f_ret_cov = f_ret_cov_B.value
  
  trials = []
  for seed in np.array(pdf.seed):
    market_condition = simulate_market(f_ret_avg, f_ret_cov, seed)
    market_features = featurize(market_condition)
    for ticker in models.keys(): 
      trial = predict(market_features, models[ticker])
      trials.append([ticker, seed, trial])
    
  # Return a dataframe with each simulation across instruments per row
  trials_pdf = pd.DataFrame(data=trials, columns=['ticker', 'seed', 'trial'])
  return trials_pdf

In [0]:
# Control experiment
to_date = trial_date.strftime("%Y-%m-%d")
from_date = (trial_date - timedelta(days = d_days)).strftime("%Y-%m-%d")
seed_init = int(trial_date.timestamp())

# create a dataframe of seeds so that each trial will result in a different simulation
# each executor is responsible for num_instruments * ( total_runs / num_executors ) trials
seed_pdf = pd.DataFrame([[seed_init + x, x % parallelism] for x in np.arange(0, runs)], columns = ['seed', 'executor'])
seed_df = spark.createDataFrame(seed_pdf).repartition(parallelism, 'executor')

# Compute volatility
market_df = retrieve_market_factors(from_date, to_date)
f_ret_avg, f_ret_cov = compute_volatility(market_df)
f_ret_avg_B = spark.sparkContext.broadcast(f_ret_avg)
f_ret_cov_B = spark.sparkContext.broadcast(f_ret_cov)

# group dataframe of seeds at the executor level and run simulations
mc_df = seed_df.groupBy('executor').apply(run_trials)

# store runs
mc_df \
  .withColumn('run_date', F.lit(to_date)) \
  .join(spark.read.table(portfolio_table), 'ticker', 'inner') \
  .select('run_date', 'ticker', 'seed', 'trial', 'industry', 'country') \
  .write \
  .partitionBy("run_date") \
  .mode("append") \
  .format("delta") \
  .saveAsTable(trial_table)

In [0]:
spark.read.table(trial_table).limit(100).toPandas()

,run_date,ticker,seed,trial,industry,country
0,2019-07-01,SCCO,1561939201,-0.010716,Industrial Metals & Mining,PERU
1,2019-07-01,AVAL,1561939201,-0.010725,Financial Services,COLOMBIA
2,2019-07-01,SIM,1561939201,-0.000772,Industrial Metals & Mining,MEXICO
3,2019-07-01,BCH,1561939201,-0.017152,Banks,CHILE
4,2019-07-01,BLX,1561939201,-0.005208,Banks,PANAMA
5,2019-07-01,CPA,1561939201,-0.027181,Travel & Leisure,PANAMA
6,2019-07-01,GRAM,1561939201,-0.013018,Construction & Materials,PERU
7,2019-07-01,EC,1561939201,0.000439,Oil & Gas Producers,COLOMBIA
8,2019-07-01,KOF,1561939201,-0.019655,Beverages,MEXICO
9,2019-07-01,ENIA,1561939201,-0.025928,Electricity,CHILE


# `HOMEWORK` store raw trials and predict returns as UDFs
Given your model packaged as `pyfunc`, one can apply prediction as a user defined function. How would you store all simulated market data and predict instruments returns through a simple SQL code? How would you store model version alongside predictions?

```
SELECT 
  m.seed, 
  m.instrument, 
  predict_return(m.conditions, m.instrument)
FROM 
  monte-carlo m
```
